## Imports and setups

Substitute variables with appropriate file paths/selections

In [ ]:
import sys
sys.path.append('/Users/isaacdaviet/Desktop/thesis/python_versions')
# replace with directory containing the .py calculation files below
import clustering_functions as clustering
import pandas as pd
import os

In [ ]:
project_name = 'Mason'
# Name of project for file saving and graph title purposes

selected_pca_plotly_folder = r'/Users/isaacdaviet/Desktop/results/PCA_analysis/PCAs_of_interest/final_pcas_of_interest'
# folder containing .csv files of umap reductions from previous step (umap_final_plot_generation.ipynb)

selected_graphs_csv = r'/Users/isaacdaviet/Desktop/graphs/clustering/mason_umap_clustering_analysis_binders.csv'
# csv file containing ranges to test/plot. column format: [distance metric, start neighbors, end neighbors, neighbors step, start dist, end dist, dist step]. INCLUDE THESE EXACT COLUMN NAMES

## Generate list of PC combos from folder containing final plotly graphs

In [ ]:
labeled_seq_csv_file = r'/Users/isaacdaviet/Desktop/mason_igfold_models/mason_sequences_label.csv'
# csv file containing sequences in column 1 and their status as binder (1)/non all (0) in the second column

selected_pca_plotly_folder = r'/Users/isaacdaviet/Desktop/results/PCA_analysis/PCAs_of_interest/final_pcas_of_interest'

pca_df = r'/Users/isaacdaviet/Desktop/results/PCA_analysis/mason-PCAdf95.csv'

pca_df_iseq = clustering.add_seqs_umap_df(pca_df, labeled_seq_csv_file)

pca_df_iseq.to_csv(pca_df, index = False)

In [ ]:
html_files = [file for file in os.listdir(selected_pca_plotly_folder) if file.endswith('.html')]

# extract pc combos from names of plotly html file names in folder
pc_combos = []
for entry in html_files:
    combo = entry.split('_')[-1]
    combo = combo.replace('.html', '')
    combo = combo.replace('PCA', '')
    combo = combo.split('-')

    pc_combos.append(combo)

column_names = ['pc1', 'pc2', 'eps', 'min_samples', 'generate']
optimization_csv_df = pd.DataFrame(columns = column_names)

# generate unique dfs for each combo
all_dfs = []
for combo in pc_combos:

    x_column = pca_df_iseq.filter(like=f'PCA{pc1}_', axis=1)
    x_axes = x_column.columns.tolist()[0]
    y_column = pca_df_iseq.filter(like=f'PCA{pc2}_', axis=1)
    y_axes = y_column.columns.tolist()[0]

    X = pd.DataFrame(pca_df_iseq, columns=[x_axes, y_axes])
    X['Labels'] = pd.DataFrame(pca_df_iseq, columns=['label'])
    X['Sequences'] = pd.DataFrame(pca_df_iseq, columns=['Sequences'])
    X['iseq'] = pd.DataFrame(pca_df_iseq, columns=['iseq'])
    all_dfs.append(X)

       PCA3_ExpVar:4.20%  PCA7_ExpVar:2.96%      Labels   Sequences   iseq
0               0.001049           0.014570  Non Binder  WPTNAMFMHS  12740
1               0.004517           0.040964  Non Binder  WSMSAFFMYM  21527
2              -0.029688          -0.035011  Non Binder  FFDTSSFAFA  21135
3               0.000650           0.044158  Non Binder  YADYGLFHIH   9127
4              -0.060888          -0.091178  Non Binder  FPMHRHYVIK  23527
...                  ...                ...         ...         ...    ...
34141          -0.045774           0.079494  Non Binder  YEAASLYAND   9973
34142          -0.025123           0.013244  Non Binder  WLIPGFFVYM  23700
34143           0.038876          -0.029588  Non Binder  WDLVRHFENP  24367
34144          -0.010265          -0.014975  Non Binder  WMESAMYPHI  10973
34145          -0.035811          -0.085658  Non Binder  FRQVDFYIHP  14581

[34146 rows x 5 columns]


In [ ]:
test_df = all_dfs[0]

clustered_df = clustering.agglomerative_clustering(test_df, n_clusters, linkage, affinity)


In [ ]:
unique_positive_fv =r'/Users/isaacdaviet/Desktop/mason_igfold_models/mason_igfold_models/outfile/mHER_H3_AgPos_unique_fv.csv'

unique_negative_fv = r'/Users/isaacdaviet/Desktop/mason_igfold_models/mason_igfold_models/outfile/mHER_H3_AgNeg_unique_fv.csv'

clustering.add_iseq_to_labels_file(labeled_seq_csv_file, unique_positive_fv, unique_negative_fv)

KeyboardInterrupt: 

# Optimize clustering


Run each umap selection individually until desired clusters are acheived. When satified, switch 'generate' column to done.

Steps:
- select which clustering algorithm to use and which point type (binders/non binders) to use
- In the 'selected_graphs_csv' file, fill the generate column with 'y' for the graphs you would like to optimize
- Fill in the variables in the csv file for the appropriate algorithm (dbscan_eps + dbscan_min_samples for dbscan and agg_n_clusters and agg_linkage for agglomerative)
- Save changes to csv file and run cell below
- Analyze resulting plotly and adjust parameters until satisified with the clustering
- Use the 'combine_clusters' column to specify which clusters, if any, should be merged. Note: specifically designed for   clusters that are difficult to cluster as a single group and that would be classified as 'low priority' in the QC steps
- When satisfied, switch 'generate' to 'done' and proceed to next graph


In [ ]:
save_folder = r'/Users/isaacdaviet/Desktop/results/clustering/PCA_dbscan_clusters'
#save folder for graphs and cvs files.

algorithm = 'dbscan'
# Which clustering algorithm to use. As it stands now, dbscan and agglomerative are the only options available.

points_to_optimize = 'binders'
# what points to show on graphs. 'all' shows binders and non binders in same graph, 'binders' show only binders, 'non binders' shows only non binders


clustering.cluster_extract_and_plot(selected_graphs_csv, labeled_seq_csv_file, umap_reductions_folder, clusters = points_to_optimize, algorithm = algorithm, generate_indicator = 'y', plt_size = [60, 20], pt_size = 30, fontsize=64, show_unclustered_pts='n' ,save_graph = 'n', save_folder = save_folder, project_name=project_name, save_clusters_csv = 'n', save_plotly = 'y')


# Full Clustering

Produces csv, plotly and png graphs of all clusters indicated by 'done' in the generate column. Png images contain original binder/non binder + binder clusters + non binder clusters graphs as a single image.

NOTE: cluster graphs in png images DO NOT display any unclustered points, but the plotly graphs do.

In [ ]:
save_folder = r'/Users/isaacdaviet/Desktop/graphs/clustering/dbscan_clusters'
#save folder for graphs and cvs files.

algorithm = 'dbscan'

### Final save (csv, html plotly and subplots)
clustering.cluster_extract_and_plot(selected_graphs_csv,
                                     labeled_seq_csv_file, umap_reductions_folder,
                                     clusters = 'both',
                                     algorithm = algorithm,
                                     generate_indicator = 'done',
                                     plt_size = [60, 20],
                                     pt_size = 20,
                                     fontsize=64,
                                     show_unclustered_pts='n',
                                     save_graph = 'n',
                                     save_folder = save_folder, project_name=project_name, save_clusters_csv = 'y',
                                     save_plotly = 'n')

# Cluster QC

Modifies cluster dataframes to create add 'adjusted_clusters', which updates the clusters to reflect the manual cluster combination specified in the combine_clusters column of the cluster_analysis csv file, and a 'priority' column that classifies each point in a cluster as high/med/low/out_of_bounds priority based on percentages used as input parameters.

In [ ]:
### Manual inspection of a specific data frame's clusters as a percentage of the total number of points for a specific LABEL (ie: (# binders in cluster / total # of binders) * 100)

clusters_df = r'/Users/isaacdaviet/Desktop/graphs/clustering/correlation_clusters/csv_files/UMAP_Mason-correlation-3-25-0.0-1-2_dbscanClusters-0.15-20.csv'

points = 'non binders'

clusters_df = pd.read_csv(clusters_df)

priorities = clustering.cluster_priority(clusters_df, high_threshold = 10, low_threshold = 1, upper_limit = None, print_percentages = points)

In [ ]:
### Automatically applies cluster adjustment and prioritization to all csv_files in a folder. Thresholds/upper limit are set as percentages

high_threshold = 10
low_threshold = 1
upper_limit = 40


clusters_csv_folder = r'/Users/isaacdaviet/Desktop/graphs/clustering/dbscan_clusters'

clustering.adjust_clusters_folder(clusters_csv_folder, selected_graphs_csv, point_type = 'Binder', high_threshold = high_threshold, low_threshold = low_threshold, upper_limit = upper_limit)

Adjusting UMAP_Mason-euclidean-3-140-0.0-2-3_dbscanClusters-0.1-30.csv
Adjusting UMAP_Mason-manhattan-5-25-0.0-3-5_dbscanClusters-0.07-20.csv
Adjusting UMAP_Mason-cosine-5-35-0.25-4-5_dbscanClusters-0.12-60.csv
Adjusting UMAP_Mason-manhattan-5-65-0.2-2-5_dbscanClusters-0.08-30.csv
Adjusting UMAP_Mason-manhattan-5-185-0.0-2-5_dbscanClusters-0.07-25.csv
Adjusting UMAP_Mason-cosine-3-95-0.1-1-3_dbscanClusters-0.1-30.csv
Adjusting UMAP_Mason-euclidean-5-140-0.0-4-5_dbscanClusters-0.05-30.csv
Adjusting UMAP_Mason-correlation-5-25-0.0-3-5_dbscanClusters-0.12-45.csv
Adjusting UMAP_Mason-correlation-3-25-0.0-1-3_dbscanClusters-0.15-30.csv
Adjusting UMAP_Mason-cosine-3-95-0.0-2-3_dbscanClusters-0.12-35.csv
Adjusting UMAP_Mason-cosine-5-95-0.0-3-4_dbscanClusters-0.09-35.csv
Adjusting UMAP_Mason-euclidean-5-45-0.0-3-4_dbscanClusters-0.09-45.csv
Adjusting UMAP_Mason-hamming-3-15-0.0-2-3_dbscanClusters-0.1-35.csv
Adjusting UMAP_Mason-correlation-3-25-0.0-2-3_dbscanClusters-0.15-30.csv
Adjusting UMA